In [62]:
# 사용할 기본 패키지 로드
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches
%matplotlib inline
import os
import numpy as np

In [63]:
Basic_dir = os.path.dirname(os.path.dirname(os.path.realpath('EDA.ipynb')))
PATH = Basic_dir + '\\Data\\Data(HeartD_B)'
os.listdir(PATH)

['sample_submission.csv', 'test.csv', 'train.csv']

In [64]:
# 결측값 없음 / 15개의 열과 151개의 행으로 이루어진 데이터 구조
df_train = pd.read_csv(PATH + '\\train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        151 non-null    int64  
 1   age       151 non-null    int64  
 2   sex       151 non-null    int64  
 3   cp        151 non-null    int64  
 4   trestbps  151 non-null    int64  
 5   chol      151 non-null    int64  
 6   fbs       151 non-null    int64  
 7   restecg   151 non-null    int64  
 8   thalach   151 non-null    int64  
 9   exang     151 non-null    int64  
 10  oldpeak   151 non-null    float64
 11  slope     151 non-null    int64  
 12  ca        151 non-null    int64  
 13  thal      151 non-null    int64  
 14  target    151 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 17.8 KB


In [65]:
df_test = pd.read_csv(PATH + '\\test.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        152 non-null    int64  
 1   age       152 non-null    int64  
 2   sex       152 non-null    int64  
 3   cp        152 non-null    int64  
 4   trestbps  152 non-null    int64  
 5   chol      152 non-null    int64  
 6   fbs       152 non-null    int64  
 7   restecg   152 non-null    int64  
 8   thalach   152 non-null    int64  
 9   exang     152 non-null    int64  
 10  oldpeak   152 non-null    float64
 11  slope     152 non-null    int64  
 12  ca        152 non-null    int64  
 13  thal      152 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 16.8 KB


In [66]:
df_train['age_gr'] = np.where(df_train.age < 20, 10, 
                     np.where(df_train.age < 30, 20, 
                     np.where(df_train.age < 40, 30, 
                     np.where(df_train.age < 50, 40,
                     np.where(df_train.age < 60, 50,
                     np.where(df_train.age < 70, 60, 70))))))
df_test['age_gr'] = np.where(df_test.age < 20, 10, 
                     np.where(df_test.age < 30, 20, 
                     np.where(df_test.age < 40, 30, 
                     np.where(df_test.age < 50, 40,
                     np.where(df_test.age < 60, 50,
                     np.where(df_test.age < 70, 60, 70))))))
df_train.drop(['id', 'age'], axis = 1, inplace = True)
df_test.drop(['id', 'age'], axis = 1, inplace = True)
df_train.head()

,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,age_gr
0,1,2,130,197,1,0,152,0,1.2,0,0,2,1,50
1,1,3,152,298,1,1,178,0,1.2,1,0,3,1,50
2,1,1,192,283,0,0,195,0,0.0,2,1,3,0,50
3,0,0,138,236,0,0,152,1,0.2,1,0,2,1,40
4,1,1,122,192,0,1,174,0,0.0,2,0,2,1,30


In [67]:
val_list1 = ['thalach', 'oldpeak', 'slope', 'ca']
transform_data1 = df_train[val_list1]
transform_data2 = df_test[val_list1] 

# 가장 일반적인 StandardScaler를 사용해서 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

std_transform_data1 = scaler.fit_transform(transform_data1)
std_transform_data2 = scaler.fit_transform(transform_data2)

df_train[transform_data1.columns] = std_transform_data1
df_test[transform_data2.columns] = std_transform_data2

df_train.head()

,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,age_gr
0,1,2,130,197,1,0,0.058612,0,0.206190,-2.158717,-0.659316,2,1,50
1,1,3,152,298,1,1,1.170264,0,0.206190,-0.591572,-0.659316,3,1,50
2,1,1,192,283,0,0,1.897112,0,-0.902462,0.975574,0.434714,3,0,50
3,0,0,138,236,0,0,0.058612,1,-0.717686,-0.591572,-0.659316,2,1,40
4,1,1,122,192,0,1,0.999240,0,-0.902462,0.975574,-0.659316,2,1,30


In [68]:
# EDA에서 선별한 변수리스트
x_list = ['thalach', 'oldpeak', 'slope', 'ca', 'sex', 'cp', 'exang', 'thal', 'age_gr']

In [69]:
# KNN 모델
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
# 정확도 테스트(훈련데이터 셋)
from sklearn.metrics import accuracy_score
n = 7
neigh_n = KNeighborsClassifier(n_neighbors = n, weights = 'distance')
neigh_n.fit(df_train[x_list], list(df_train.target))
y_pred = neigh_n.predict(df_train[x_list])
accuracy = accuracy_score(df_train.target, y_pred)
# print(classification_report(y_train, y_pred))
print(f'Accuracy : {accuracy*100}')

Accuracy : 100.0


In [70]:
# 정답파일 작성
predict = pd.read_csv(PATH + '\\sample_submission.csv')
predict
len(predict), len(df_test)
predict['target'] = neigh_n.predict(df_test[x_list])
predict.to_csv(PATH + 'sample_submission(KNN).csv', index = False)
pd.read_csv(PATH + 'sample_submission(KNN).csv').head()

,id,target
0,1,0
1,2,1
2,3,1
3,4,0
4,5,1


In [71]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import classification_report, confusion_matrix
model = DecisionTreeClassifier()
model.fit(df_train[x_list], list(df_train.target))

DecisionTreeClassifier()

In [73]:
# 학습데이터에 대한 성능평가
# print(confusion_matrix(y_train, model.predict(X_train)))
print(classification_report(df_train.target, model.predict(df_train[x_list])))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        68
           1       1.00      1.00      1.00        83

    accuracy                           1.00       151
   macro avg       1.00      1.00      1.00       151
weighted avg       1.00      1.00      1.00       151



In [76]:
# 정답파일 작성
predict = pd.read_csv(PATH + '\\sample_submission.csv')
predict.target = model.predict(df_test[x_list])
predict.to_csv(PATH + 'sample_submission(DT).csv', index = False)
pd.read_csv(PATH + 'sample_submission(DT).csv').head()

,id,target
0,1,0
1,2,1
2,3,1
3,4,0
4,5,1
